# Oil Production Toy Problems

These problems illustrate the use of oil production cost data in the openLCA database and the construction of more complex optimisation problems using openLCA data.

# Problem Statement

An oil company wishes to find the cheapest region to extract 10000 tonnes of crude oil from onshore oil plants in the Middle East, Russia or Africa and then transport the oil to a nearby refinery by sea tanker. The distance from the extraction site to the refinery in each region is given in the table below.

Region | Distance / km
-- | --
Middle East | 200
Russia | 600
Africa | 400

## Assumptions

* We ignore the environmental impact of extraction, transport and building the refinery.
* We ignore the cost of building the refinery.

# Analysis

First we find the three crude oil production processes in the openLCA database.

In [1]:
import mola
import mola.dataimport as di
import mola.dataview as dv
conn = di.get_sqlite_connection()
mola.set_option('show.SQL', False)

In [2]:
oil_production_dfr = dv.get_processes(conn, name=['petroleum production, onshore%'])
oil_production_dfr

SELECT "TBL_PROCESSES"."ID","TBL_PROCESSES"."REF_ID","TBL_PROCESSES"."NAME","TBL_PROCESSES"."PROCESS_TYPE","TBL_LOCATIONS"."NAME" "LOCATION","TBL_PROCESSES"."F_QUANTITATIVE_REFERENCE" FROM "TBL_PROCESSES" LEFT JOIN "TBL_LOCATIONS" ON CAST("TBL_PROCESSES"."F_LOCATION" AS INT)="TBL_LOCATIONS"."ID" WHERE "TBL_PROCESSES"."NAME" LIKE 'petroleum production, onshore%'


,ID,REF_ID,NAME,PROCESS_TYPE,LOCATION,F_QUANTITATIVE_REFERENCE
0,7076520,81d22f90-89fc-3f04-845e-97615867d8f9,"petroleum production, onshore | petroleum | AP...",LCI_RESULT,Russian Federation,7076522
1,10012499,2c08823a-4575-30ba-bf61-36e2fd4d56e6,"petroleum production, onshore | petroleum | AP...",LCI_RESULT,Rest-of-World,10012501
2,15742513,c1827905-bfad-3fa1-b00b-1303f970ae92,"petroleum production, onshore | petroleum | AP...",LCI_RESULT,Middle East,15742515
3,29432025,cd177b7d-e908-3e69-b40c-4827b4abaa4d,"petroleum production, onshore | petroleum | AP...",LCI_RESULT,Africa,29432027


We can explicitly find the cost of producing oil at the three locations using the following `mola` function.

In [3]:
production_ref_id = [
    '81d22f90-89fc-3f04-845e-97615867d8f9',
    'c1827905-bfad-3fa1-b00b-1303f970ae92',
    'cd177b7d-e908-3e69-b40c-4827b4abaa4d'
]
dv.get_process_product_flow_costs(conn, process_ref_ids=production_ref_id)

,PROCESS_REF_ID,PROCESS_NAME,LOCATION,FLOW_REF_ID,FLOW_NAME,COST_VALUE,CURRENCY,UNITS
0,81d22f90-89fc-3f04-845e-97615867d8f9,"petroleum production, onshore | petroleum | AP...",Russian Federation,e6aad2de-0b1b-49c3-a0c4-797ba34d87e5,petroleum,0.192,Euro,kg
1,c1827905-bfad-3fa1-b00b-1303f970ae92,"petroleum production, onshore | petroleum | AP...",Middle East,e6aad2de-0b1b-49c3-a0c4-797ba34d87e5,petroleum,0.192,Euro,kg
2,cd177b7d-e908-3e69-b40c-4827b4abaa4d,"petroleum production, onshore | petroleum | AP...",Africa,e6aad2de-0b1b-49c3-a0c4-797ba34d87e5,petroleum,0.192,Euro,kg


OpenLCA reports the cost is the same in each country so it would appear to be an assumption. The same pattern is seen for costs for other processes in openLCA when one looks across location.

In openLCA there is only only one global transport process for petroleum by sea tanker. We obtain the transport process from the database below.

In [4]:
oil_transport_dfr = dv.get_processes(conn, name=['transport, freight, sea, tanker for petroleum %'])
oil_transport_dfr

SELECT "TBL_PROCESSES"."ID","TBL_PROCESSES"."REF_ID","TBL_PROCESSES"."NAME","TBL_PROCESSES"."PROCESS_TYPE","TBL_LOCATIONS"."NAME" "LOCATION","TBL_PROCESSES"."F_QUANTITATIVE_REFERENCE" FROM "TBL_PROCESSES" LEFT JOIN "TBL_LOCATIONS" ON CAST("TBL_PROCESSES"."F_LOCATION" AS INT)="TBL_LOCATIONS"."ID" WHERE "TBL_PROCESSES"."NAME" LIKE 'transport, freight, sea, tanker for petroleum %'


,ID,REF_ID,NAME,PROCESS_TYPE,LOCATION,F_QUANTITATIVE_REFERENCE
0,33780409,8d82d02e-f52f-41f9-aa10-ae607746b04e,"transport, freight, sea, tanker for petroleum ...",LCI_RESULT,Global,33780411


In [5]:
transport_ref_id = [
    '8d82d02e-f52f-41f9-aa10-ae607746b04e'
]
dv.get_process_product_flow_costs(conn, process_ref_ids=transport_ref_id)

,PROCESS_REF_ID,PROCESS_NAME,LOCATION,FLOW_REF_ID,FLOW_NAME,COST_VALUE,CURRENCY,UNITS
0,8d82d02e-f52f-41f9-aa10-ae607746b04e,"transport, freight, sea, tanker for petroleum ...",Global,774bc814-70cf-4389-8bf8-e6435174c72e,"transport, freight, sea, tanker for petroleum",0.000314,Euro,t*km


The optimal region is the Middle East, since the cost of production is the same according to openLCA in each region and the Middle East has the shortest distance to the refinery. Notice that the cost of extraction of a tonne of oil (192 Euros) is far larger than the cost of transporting it over 1000km by sea (0.314 Euros).

## Sets and Parameters

In this section we build the sets and parameters for `mola`. First we load the specification.

In [6]:
import mola.specification5 as ms
from importlib import reload
spec = ms.GeneralSpecification()

Then we set up a configuration file for the sets.

In [7]:
import mola.input as mi
cost_set_file_name = 'Configuration/cost_set_data.json'
user_sets = mi.get_model_user_sets(spec, cost_set_file_name)
user_sets

Model sets saved


{'F_m': ['e6aad2de-0b1b-49c3-a0c4-797ba34d87e5'],
 'F_s': [],
 'F_t': ['774bc814-70cf-4389-8bf8-e6435174c72e'],
 'D': ['d1'],
 'T': ['t1'],
 'K': ['k1'],
 'P_m': ['81d22f90-89fc-3f04-845e-97615867d8f9',
  'c1827905-bfad-3fa1-b00b-1303f970ae92',
  'cd177b7d-e908-3e69-b40c-4827b4abaa4d'],
 'P_t': ['8d82d02e-f52f-41f9-aa10-ae607746b04e'],
 'P_s': [],
 'KPI': [],
 'OBJ': ['environment', 'cost'],
 'F': [],
 'P': []}

In [8]:
import mola.dataview as dv
import mola.dataimport as di
import pandas as pd
from pyomo.environ import *
conn = di.get_sqlite_connection()
lookup = dv.LookupTables(conn)

We show a widget that loads and saves the user configuration of sets in a tabbed interface. Existing user configuration data is loaded if there is saved model data file.

In [9]:
import mola.widgets as mw
lookups = {'F_m': lookup.get('flows'), 'P_m': lookup.get('processes'), 'F_t': lookup.get('flows'), 
           'P_t': lookup.get('processes'), 'F_s': lookup.get('flows'), 
           'P_s': lookup.get('processes'), 'KPI': lookup.get('KPI')}
vbox, tab = mw.get_sets(spec, lookups, cost_set_file_name)
vbox

Below, we load the sets defined from the JSON file and incorporate any non-user defined sets.

In [10]:
set_data = mi.get_model_user_sets(spec, cost_set_file_name)
set_data

Model sets saved


{'F_m': ['e6aad2de-0b1b-49c3-a0c4-797ba34d87e5'],
 'F_s': [],
 'F_t': ['774bc814-70cf-4389-8bf8-e6435174c72e'],
 'D': ['d1'],
 'T': ['t1'],
 'K': ['k1'],
 'P_m': ['81d22f90-89fc-3f04-845e-97615867d8f9',
  'c1827905-bfad-3fa1-b00b-1303f970ae92',
  'cd177b7d-e908-3e69-b40c-4827b4abaa4d'],
 'P_t': ['8d82d02e-f52f-41f9-aa10-ae607746b04e'],
 'P_s': [],
 'KPI': [],
 'OBJ': ['environment', 'cost'],
 'F': [],
 'P': []}

The parameters are defined using the widget below.

In [11]:
import qgrid
cost_parameters_file_name = 'Configuration/cost_parameters_data.json'
param_dfr, param_dict = mi.get_model_user_parameters(spec, set_data, cost_parameters_file_name)
mw.get_parameters(param_dfr, cost_parameters_file_name)

## Model build

To build the model we need to retrieve the cost of each flow from the database. This is done using the `populate` method
of the `Specification` object.

In [12]:
json_files = [cost_set_file_name, cost_parameters_file_name]
model_instance = spec.populate(json_files)

We can see that the cost data for the product flows has been loaded from the openLCA database.

In [13]:
model_instance.phi.pprint()

phi : Size=8, Index=phi_index, Domain=Any, Default=0, Mutable=False
    Key                                                                                    : Value
    ('774bc814-70cf-4389-8bf8-e6435174c72e', '8d82d02e-f52f-41f9-aa10-ae607746b04e', 't1') : 0.000313973464772506
    ('e6aad2de-0b1b-49c3-a0c4-797ba34d87e5', '81d22f90-89fc-3f04-845e-97615867d8f9', 't1') :                0.192
    ('e6aad2de-0b1b-49c3-a0c4-797ba34d87e5', 'c1827905-bfad-3fa1-b00b-1303f970ae92', 't1') :                0.192
    ('e6aad2de-0b1b-49c3-a0c4-797ba34d87e5', 'cd177b7d-e908-3e69-b40c-4827b4abaa4d', 't1') :                0.192


# Solution

We deactivate environment and combined objectives.

In [14]:
model_instance.Environmental_Impact.deactivate()
model_instance.Cost.activate()
model_instance.Environmental_Cost_Impact.deactivate()

In [15]:
opt = SolverFactory("glpk")
results = opt.solve(model_instance)
results.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 1920627.94692954
  Upper bound: 1920627.94692954
  Number of objectives: 1
  Number of constraints: 8
  Number of variables: 9
  Number of nonzeros: 18
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 1
      Number of created subproblems: 1
  Error rc: 0
  Time: 0.019832372665405273
# ----------------------------------------------------------
#   Solution Information
# -------------

In [16]:
import mola.output as mo
pd.set_option('display.max_colwidth', 1000)
mo.get_entity(model_instance.Flow, lookup, units=['P_m'])

,Flow,Units,F_m,P_m,K,T
0,0.0,kg,petroleum,"petroleum production, onshore | petroleum | APOS, S | Russian Federation",k1,t1
1,10000000.0,kg,petroleum,"petroleum production, onshore | petroleum | APOS, S | Middle East",k1,t1
2,0.0,kg,petroleum,"petroleum production, onshore | petroleum | APOS, S | Africa",k1,t1


In [17]:
mo.get_entity(model_instance.Specific_Material_Transport_Flow, lookup, units=['P_m'])

,Specific_Material_Transport_Flow,Units,F_m,P_m,F_t,P_t,K,T
0,-0.0,kg,petroleum,"petroleum production, onshore | petroleum | APOS, S | Russian Federation","transport, freight, sea, tanker for petroleum","transport, freight, sea, tanker for petroleum | transport, freight, sea, tanker for petroleum | APOS, S | Global",k1,t1
1,10000000.0,kg,petroleum,"petroleum production, onshore | petroleum | APOS, S | Middle East","transport, freight, sea, tanker for petroleum","transport, freight, sea, tanker for petroleum | transport, freight, sea, tanker for petroleum | APOS, S | Global",k1,t1
2,-0.0,kg,petroleum,"petroleum production, onshore | petroleum | APOS, S | Africa","transport, freight, sea, tanker for petroleum","transport, freight, sea, tanker for petroleum | transport, freight, sea, tanker for petroleum | APOS, S | Global",k1,t1


In [18]:
mo.get_entity(model_instance.Specific_Transport_Flow, lookup, units=True)

,Specific_Transport_Flow,Units,F_t,P_t,K,T
0,2000000.0,t*km,"transport, freight, sea, tanker for petroleum","transport, freight, sea, tanker for petroleum | transport, freight, sea, tanker for petroleum | APOS, S | Global",k1,t1


In [19]:
mo.get_entity(model_instance.Cost)

,Objective
0,1.920628e+06


# Environment Impact and Cost

In this section we consider both the environment impact and the cost of extracting and transporting the oil.

We augment the cost sets by choosing the ReCiPe Midpoint method and the fossil depletion category.

In [20]:
env_set_file_name = 'Configuration/env_set_data.json'
vbox, tab = mw.get_sets(spec, lookups, env_set_file_name)
vbox

We set both objective weights to one meaning that we value 1 kg Oil Eq. as much as 1 Euro.

In [21]:
env_parameters_file_name = 'Configuration/env_parameters_data.json'
env_set_data = mi.get_model_user_sets(spec, env_set_file_name)
env_set_data

Model sets saved


{'F_m': ['e6aad2de-0b1b-49c3-a0c4-797ba34d87e5'],
 'F_s': [],
 'F_t': ['774bc814-70cf-4389-8bf8-e6435174c72e'],
 'D': ['d1'],
 'T': ['t1'],
 'K': ['k1'],
 'P_m': ['81d22f90-89fc-3f04-845e-97615867d8f9',
  'c1827905-bfad-3fa1-b00b-1303f970ae92',
  'cd177b7d-e908-3e69-b40c-4827b4abaa4d'],
 'P_t': ['8d82d02e-f52f-41f9-aa10-ae607746b04e'],
 'P_s': [],
 'KPI': ['bf42d8f0-23fc-32f3-b159-f79bfccfec28'],
 'OBJ': ['environment', 'cost'],
 'F': [],
 'P': []}

In [22]:
param_dfr, param_dict = mi.get_model_user_parameters(spec, env_set_data, env_parameters_file_name)
mw.get_parameters(param_dfr, env_parameters_file_name)

Then we rebuild the model.

In [23]:
json_files = [env_set_file_name, env_parameters_file_name]
model_instance2 = spec.populate(json_files)

First we activate the environmental and cost objectives in the model instance and solve.

In [24]:
model_instance2.Environmental_Impact.deactivate()
model_instance2.Cost.deactivate()
model_instance2.Environmental_Cost_Impact.activate()
results = opt.solve(model_instance2)
results.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 12688000.1655666
  Upper bound: 12688000.1655666
  Number of objectives: 1
  Number of constraints: 8
  Number of variables: 9
  Number of nonzeros: 18
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 1
      Number of created subproblems: 1
  Error rc: 0
  Time: 0.02035355567932129
# ----------------------------------------------------------
#   Solution Information
# --------------

In [25]:
mo.get_entity(model_instance.Flow, lookup, units=True)

,Flow,Units,F_m,P_m,K,T
0,0.0,kg,petroleum,"petroleum production, onshore | petroleum | APOS, S | Russian Federation",k1,t1
1,10000000.0,kg,petroleum,"petroleum production, onshore | petroleum | APOS, S | Middle East",k1,t1
2,0.0,kg,petroleum,"petroleum production, onshore | petroleum | APOS, S | Africa",k1,t1


In [26]:
mo.get_entity(model_instance2.Specific_Material_Transport_Flow, lookup, units=True)

,Specific_Material_Transport_Flow,Units,F_m,P_m,F_t,P_t,K,T
0,-0.0,kg,petroleum,"petroleum production, onshore | petroleum | APOS, S | Russian Federation","transport, freight, sea, tanker for petroleum","transport, freight, sea, tanker for petroleum | transport, freight, sea, tanker for petroleum | APOS, S | Global",k1,t1
1,10000000.0,kg,petroleum,"petroleum production, onshore | petroleum | APOS, S | Middle East","transport, freight, sea, tanker for petroleum","transport, freight, sea, tanker for petroleum | transport, freight, sea, tanker for petroleum | APOS, S | Global",k1,t1
2,-0.0,kg,petroleum,"petroleum production, onshore | petroleum | APOS, S | Africa","transport, freight, sea, tanker for petroleum","transport, freight, sea, tanker for petroleum | transport, freight, sea, tanker for petroleum | APOS, S | Global",k1,t1


In [27]:
mo.get_entity(model_instance2.Specific_Transport_Flow, lookup, units=True)

,Specific_Transport_Flow,Units,F_t,P_t,K,T
0,2000000.0,t*km,"transport, freight, sea, tanker for petroleum","transport, freight, sea, tanker for petroleum | transport, freight, sea, tanker for petroleum | APOS, S | Global",k1,t1


In [28]:
mo.get_entity(model_instance2.Environmental_Cost_Impact, lookup, units=True)

,Objective
0,1.268800e+07


We confirm that the sum of the environmental and cost objectives is equal to the overall wieghted objective below.

In [29]:
print(value(model_instance2.Environmental_Impact['bf42d8f0-23fc-32f3-b159-f79bfccfec28']) + value(model_instance2.Cost))
print(value(model_instance2.Environmental_Cost_Impact))

12688000.165566565
12688000.165566565


# Oil Refinery

In this section we add an oil refinery to the problem as a separate task. To start with we set this problem up with a total demand. Subsequently, we connect the oil production task to this task and remove the total demand parameter from the production problem.

## Problem Statement

An oil company wishes to find the cheapest means to transport 5000 tonnes of diesel from an oil refinery by using either freight train or sea tanker to a manufacturing plant. The distance from the refinery to the manufacturing plant is 500km.

### Assumptions

* We suppose we can use the Rest-of-World diesel production process from openLCA in this problem. Its product flow is 1kg of diesel oil. 
* The refinery uses 4kg of crude oil to produce 1kg of diesel. 

## Sets and Parameters

In [30]:
refinery_set_file_name = 'Configuration/refinery_set_data.json'
vbox, tab = mw.get_sets(spec, lookups, refinery_set_file_name)
vbox

In [31]:
refinery_parameters_file_name = 'Configuration/refinery_parameters_data.json'
refinery_set_data = mi.get_model_user_sets(spec, refinery_set_file_name)
refinery_set_data

Model sets saved


{'F_m': ['e6aad2de-0b1b-49c3-a0c4-797ba34d87e5',
  '291fc06d-1b3e-4077-aabb-346b588ed24b'],
 'F_s': [],
 'F_t': ['774bc814-70cf-4389-8bf8-e6435174c72e',
  '0ace02fa-eca5-482d-a829-c18e46a52db4'],
 'D': ['d1'],
 'T': ['t1'],
 'K': ['k1', 'k2'],
 'P_m': ['81d22f90-89fc-3f04-845e-97615867d8f9',
  'c1827905-bfad-3fa1-b00b-1303f970ae92',
  'cd177b7d-e908-3e69-b40c-4827b4abaa4d',
  '5d9ef634-f3ca-4ec5-8315-4c07d736b4ce'],
 'P_t': ['8d82d02e-f52f-41f9-aa10-ae607746b04e',
  'f615ae37-fa6d-4a47-958c-23234c986400'],
 'P_s': [],
 'KPI': ['bf42d8f0-23fc-32f3-b159-f79bfccfec28'],
 'OBJ': ['environment', 'cost'],
 'F': [],
 'P': []}

In [32]:
param_dfr, param_dict = mi.get_model_user_parameters(spec, refinery_set_data, refinery_parameters_file_name)
mw.get_parameters(param_dfr, refinery_parameters_file_name)

In [33]:
json_files = [refinery_set_file_name, refinery_parameters_file_name]
model_instance3 = spec.populate(json_files)

In [34]:
model_instance3.Environmental_Impact.deactivate()
model_instance3.Cost.activate()
model_instance3.Environmental_Cost_Impact.deactivate()
results = opt.solve(model_instance3)
results.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 1850784.93366193
  Upper bound: 1850784.93366193
  Number of objectives: 1
  Number of constraints: 31
  Number of variables: 37
  Number of nonzeros: 50
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 1
      Number of created subproblems: 1
  Error rc: 0
  Time: 0.020902395248413086
# ----------------------------------------------------------
#   Solution Information
# -----------

In [35]:
mo.get_entity(model_instance3.Flow, lookup, units=True, non_zero=True)

,Flow,Units,F_m,P_m,K,T
15,5000000.0,kg,diesel,"diesel production, petroleum refinery operation | diesel | APOS, S | Rest-of-World",k2,t1


In [36]:
mo.get_entity(model_instance3.Specific_Material_Transport_Flow, lookup, units=True, non_zero=True, distinct_levels=True)

,Specific_Material_Transport_Flow,Units,F_m,P_m,F_t,P_t,K
7,5000000.0,kg,diesel,"diesel production, petroleum refinery operation | diesel | APOS, S | Rest-of-World","transport, freight, sea, tanker for petroleum","transport, freight, sea, tanker for petroleum | transport, freight, sea, tanker for petroleum | APOS, S | Global",k2


In [37]:
mo.get_entity(model_instance3.Specific_Transport_Flow, lookup, units=True, non_zero=True, distinct_levels=True)

,Specific_Transport_Flow,Units,F_t,P_t,K
1,2500000.0,t*km,"transport, freight, sea, tanker for petroleum","transport, freight, sea, tanker for petroleum | transport, freight, sea, tanker for petroleum | APOS, S | Global",k2


# Connecting Tasks

We connect the task of oil extraction and transport, and the task of oil refinement to diesel and then transport to a manufacturer together. 

We modify the sets and parameters in memory and then write out the resulting JSON files to disk. First we copy the sets and parameters.

In [38]:
import json
multitask_sets_data = refinery_set_data.copy()
with open(refinery_parameters_file_name) as fp:
    multitask_parameters_data = json.load(fp)
multitask_parameters_data['Total_Demand']

[{'index': ['d1', 'k1'], 'value': 0.0},
 {'index': ['d1', 'k2'], 'value': 5000.0}]

We want to connect the transport of petroleum in the three regions by sea tanker in task $k_1$ to the material flow of diesel in task $k_2$. Thus we need to add one of the following constraints

$$
f_{f_{\text{petroleum}}, p_{\text{RUS_petroleum}}, f_{\text{sea tanker}}, p_{\text{sea tanker}}, k_1, t_1} + 
f_{f_{\text{petroleum}}, p_{\text{ME_petroleum}}, f_{\text{sea tanker}}, p_{\text{sea tanker}}, k_1, t_1} +
f_{f_{\text{petroleum}}, p_{\text{AF_petroleum}}, f_{\text{sea tanker}}, p_{\text{sea tanker}}, k_1, t_1}
\geq 4 Flow_{f_{\text{diesel}}, p_{\text{diesel}}, k_2, t_1}
$$


$$
Flow_{f_{\text{petroleum}}, p_{\text{RUS_petroleum}}, k_1, t_1} +
Flow_{f_{\text{petroleum}}, p_{\text{ME_petroleum}}, k_1, t_1} +
Flow_{f_{\text{petroleum}}, p_{\text{AF_petroleum}}, k_1, t_1} +
\geq 4 Flow_{f_{\text{diesel}}, p_{\text{diesel}}, k_2, t_1}
$$

In [39]:
lookup.get('P_m', list(model_instance3.P_m))

,PROCESS_NAME,LOCATION_NAME
P_t,,
81d22f90-89fc-3f04-845e-97615867d8f9,"petroleum production, onshore | petroleum | APOS, S",Russian Federation
c1827905-bfad-3fa1-b00b-1303f970ae92,"petroleum production, onshore | petroleum | APOS, S",Middle East
cd177b7d-e908-3e69-b40c-4827b4abaa4d,"petroleum production, onshore | petroleum | APOS, S",Africa
5d9ef634-f3ca-4ec5-8315-4c07d736b4ce,"diesel production, petroleum refinery operation | diesel | APOS, S",Rest-of-World


In [40]:
f_pet = 'e6aad2de-0b1b-49c3-a0c4-797ba34d87e5'
p_RUS = '81d22f90-89fc-3f04-845e-97615867d8f9'
p_ME = 'c1827905-bfad-3fa1-b00b-1303f970ae92'
p_AF = 'cd177b7d-e908-3e69-b40c-4827b4abaa4d'
f_st = '774bc814-70cf-4389-8bf8-e6435174c72e'
p_st = '8d82d02e-f52f-41f9-aa10-ae607746b04e'
f_die = '291fc06d-1b3e-4077-aabb-346b588ed24b'
p_die = '5d9ef634-f3ca-4ec5-8315-4c07d736b4ce'

Clear the task link coefficients.

In [41]:
for item in multitask_parameters_data['calA']:
    item['value'] = 0
for item in multitask_parameters_data['calB']:
    item['value'] = 0
for item in multitask_parameters_data['calC']:
    item['value'] = 0

In [42]:
for item in multitask_parameters_data['calC']:
    if item['index'][0:5] in [[f_pet, p_RUS, f_st, p_st, 'k1'], [f_pet, p_ME, f_st, p_st, 'k1'], [f_pet, p_AF, f_st, p_st, 'k1']]:
        item['value'] = 1   
# for item in multitask_parameters_data['calA']:
#     if item['index'][0:3] in [[f_pet, p_RUS, 'k1'], [f_pet, p_ME, 'k1'], [f_pet, p_AF, 'k1']]:
#         item['value'] = 1   

In [43]:
for item in multitask_parameters_data['calA']:
    if item['index'][0:3] == [f_die, p_die, 'k2']:
        item['value'] = 4

The non-zero coefficients in the task link constraint are shown below.

In [44]:
for item in multitask_parameters_data['calC']:
    if item['value']:
        print('calC', item['index'], ' = ', item['value'], sep='')
for item in multitask_parameters_data['calA']:
    if item['value']:
        print('calA', item['index'], ' = ', item['value'], sep='')

calC['e6aad2de-0b1b-49c3-a0c4-797ba34d87e5', '81d22f90-89fc-3f04-845e-97615867d8f9', '774bc814-70cf-4389-8bf8-e6435174c72e', '8d82d02e-f52f-41f9-aa10-ae607746b04e', 'k1', 't1'] = 1
calC['e6aad2de-0b1b-49c3-a0c4-797ba34d87e5', 'c1827905-bfad-3fa1-b00b-1303f970ae92', '774bc814-70cf-4389-8bf8-e6435174c72e', '8d82d02e-f52f-41f9-aa10-ae607746b04e', 'k1', 't1'] = 1
calC['e6aad2de-0b1b-49c3-a0c4-797ba34d87e5', 'cd177b7d-e908-3e69-b40c-4827b4abaa4d', '774bc814-70cf-4389-8bf8-e6435174c72e', '8d82d02e-f52f-41f9-aa10-ae607746b04e', 'k1', 't1'] = 1
calA['291fc06d-1b3e-4077-aabb-346b588ed24b', '5d9ef634-f3ca-4ec5-8315-4c07d736b4ce', 'k2', 't1'] = 4


We save these new parameters to JSON files.

In [45]:
tasks_set_file_name = 'Configuration/tasks_set_data.json'
with open(tasks_set_file_name, 'w') as fp:
    json.dump(multitask_sets_data, fp, indent=4)
tasks_parameter_file_name = 'Configuration/tasks_parameter_data.json'
with open(tasks_parameter_file_name, 'w') as fp:
    json.dump(multitask_parameters_data, fp, indent=4)

Next we rebuild the model.

In [46]:
json_files = [tasks_set_file_name, tasks_parameter_file_name]
model_instance4 = spec.populate(json_files)

A simple task chain constraints is built into the abstract model. Its concrete form is shown below.

In [47]:
model_instance4.task_chain_constraint.activate()
model_instance4.task_chain_constraint.pprint()

task_chain_constraint : Size=1, Index=task_chain_constraint_index, Active=True
    Key          : Lower : Body                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              : Upper : Active
    ('k2', 't1') :  -Inf : Specific_Material_Transport_Flow[e6aad2de-0b1b-49c3-a0c4-797ba34d87e5,81d22f90-89fc-3f04-845e-97615867d8f9,774bc814-70cf-4389-8bf8-e6435174c72e,8d82d02e-f52f-41f9-aa10-ae607746b04e,k1,t1] + Sp

In [48]:
model_instance4.Environmental_Impact.deactivate()
model_instance4.Cost.activate()
model_instance4.Environmental_Cost_Impact.deactivate()
results = opt.solve(model_instance4)
results.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 1850784.93366193
  Upper bound: 1850784.93366193
  Number of objectives: 1
  Number of constraints: 32
  Number of variables: 37
  Number of nonzeros: 54
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 1
      Number of created subproblems: 1
  Error rc: 0
  Time: 0.021797895431518555
# ----------------------------------------------------------
#   Solution Information
# -----------

In [49]:
mo.get_entity(model_instance4.Flow, lookup, units=True, non_zero=True)

,Flow,Units,F_m,P_m,K,T
15,5000000.0,kg,diesel,"diesel production, petroleum refinery operation | diesel | APOS, S | Rest-of-World",k2,t1


In [50]:
mo.get_entity(model_instance4.Specific_Material_Transport_Flow, lookup, units=True, non_zero=True, distinct_levels=True)

,Specific_Material_Transport_Flow,Units,F_m,P_m,F_t,P_t,K
7,5000000.0,kg,diesel,"diesel production, petroleum refinery operation | diesel | APOS, S | Rest-of-World","transport, freight, sea, tanker for petroleum","transport, freight, sea, tanker for petroleum | transport, freight, sea, tanker for petroleum | APOS, S | Global",k2


In [51]:
mo.get_entity(model_instance4.Specific_Transport_Flow, lookup, units=True, non_zero=True, distinct_levels=True)

,Specific_Transport_Flow,Units,F_t,P_t,K
1,2500000.0,t*km,"transport, freight, sea, tanker for petroleum","transport, freight, sea, tanker for petroleum | transport, freight, sea, tanker for petroleum | APOS, S | Global",k2


In [52]:
value(model_instance4.Cost)

1850784.9336619312

# Task implementation in Mola

We can implement tasks in `Mola` using Pyomo nodes and arcs in the `pyomo.network` package. The advantage is that we can allow the user to decide how to link tasks together rather than specify different forms of constraint in the abstract model. We can also build the front-end around higher level constructs and avoid the user having to specify a large numbers of parameters.

First we deactivate the explicit task link constraint and re-run the optimisation.

In [53]:
model_instance4.task_chain_constraint.deactivate()
results = opt.solve(model_instance4)
results.write()
mo.get_entity(model_instance4.Flow, lookup, units=True, non_zero=True, distinct_levels=True)

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 1850784.93366193
  Upper bound: 1850784.93366193
  Number of objectives: 1
  Number of constraints: 31
  Number of variables: 37
  Number of nonzeros: 50
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 1
      Number of created subproblems: 1
  Error rc: 0
  Time: 0.021445274353027344
# ----------------------------------------------------------
#   Solution Information
# -----------

,Flow,Units,F_m,P_m,K
15,5000000.0,kg,diesel,"diesel production, petroleum refinery operation | diesel | APOS, S | Rest-of-World",k2


Pyomo can generate the constraints given the nodes and arcs. We can also add expressions to the ports to deal with flow conversion between tasks.

In [54]:
from pyomo.network import *

We create a port for each task $k$.

In [57]:
def test_port_rule(model, k, t):
    d = {}
    d['flow'] = sum(model.calA[fm, pm, k, t] * model.Flow[fm, pm, k, t] for fm in model.F_m
                    for pm in model.P_m)
    d['flow'] += sum(model.calC[fm, pm, ft, pt, k, t] * 
                     model.Specific_Material_Transport_Flow[fm, pm, ft, pt, k, t] for fm in model.F_m
                     for pm in model.P_m for ft in model.F_t for pt in model.P_t) 
    return d
model_instance4.test_port = Port(model_instance4.K, model_instance4.T, rule=test_port_rule)

A pyomo arc links the two ports together.

In [58]:
model_instance4.arc = Arc(source=model_instance4.test_port['k1', 't1'], 
                          destination=model_instance4.test_port['k2', 't1']) # directed

Expanding the arc generates an equality constraint between the flow variables.

In [59]:
TransformationFactory("network.expand_arcs").apply_to(model_instance4)

The constraint is generated in a pyomo `block` called `arc_expanded`.

In [60]:
model_instance4.arc_expanded.pprint()

arc_expanded : Size=1, Index=None, Active=True
    1 Constraint Declarations
        flow_equality : Size=1, Index=None, Active=True
            Key  : Lower : Body                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              : Upper : Active
            None :   0.0 : Specific_Material_Transport_Flow[e6aad2de-0b1b-49c3-a0c4-797ba34d87e5,81d22f90-89fc-3f04-845e-97615867d8f9,774bc814-70cf-4389-8bf8-e643517

We can re-run the optimisation and obtain the same results as before.

In [61]:
results = opt.solve(model_instance4)
results.write()
mo.get_entity(model_instance4.Flow, lookup, units=True, non_zero=True, distinct_levels=True)

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 5692040.82752102
  Upper bound: 5692040.82752102
  Number of objectives: 1
  Number of constraints: 32
  Number of variables: 37
  Number of nonzeros: 54
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 1
      Number of created subproblems: 1
  Error rc: 0
  Time: 0.023131370544433594
# ----------------------------------------------------------
#   Solution Information
# -----------

,Flow,Units,F_m,P_m,K
2,20000000.0,kg,petroleum,"petroleum production, onshore | petroleum | APOS, S | Middle East",k1
15,5000000.0,kg,diesel,"diesel production, petroleum refinery operation | diesel | APOS, S | Rest-of-World",k2


In [62]:
value(model_instance4.Cost)

5692040.8275210215

# Specification of Tasks

In this problem we solve the multi-task optimisation again, but this time use the built-in infrastructure for Ports and Arcs built into the `GeneralSpecification`.

In [63]:
model_instance5 = spec.populate(json_files)
model_instance5.port.pprint()

port : Size=2, Index=port_index
    Key          : Name : Size : Variable
    ('k1', 't1') : flow :    1 : Specific_Material_Transport_Flow[e6aad2de-0b1b-49c3-a0c4-797ba34d87e5,81d22f90-89fc-3f04-845e-97615867d8f9,774bc814-70cf-4389-8bf8-e6435174c72e,8d82d02e-f52f-41f9-aa10-ae607746b04e,k1,t1] + Specific_Material_Transport_Flow[e6aad2de-0b1b-49c3-a0c4-797ba34d87e5,c1827905-bfad-3fa1-b00b-1303f970ae92,774bc814-70cf-4389-8bf8-e6435174c72e,8d82d02e-f52f-41f9-aa10-ae607746b04e,k1,t1] + Specific_Material_Transport_Flow[e6aad2de-0b1b-49c3-a0c4-797ba34d87e5,cd177b7d-e908-3e69-b40c-4827b4abaa4d,774bc814-70cf-4389-8bf8-e6435174c72e,8d82d02e-f52f-41f9-aa10-ae607746b04e,k1,t1]
    ('k2', 't1') : flow :    1 : 4*Flow[291fc06d-1b3e-4077-aabb-346b588ed24b,5d9ef634-f3ca-4ec5-8315-4c07d736b4ce,k2,t1]
